# Cerebrum Quickstart Guide

This notebook demonstrates how to use Cerebrum to access an LLM via:
- **Raw HTTP requests**
- **Langchain integration**

We'll start with a simple chat, followed by an example that uses a tool function in the conversation.

See [ENVIRONMENT_SETUP.md](ENVIRONMENT_SETUP.md) for API key info, limits, and usage guidelines.

*For reference, here is the CURL call:*

```
curl --location 'https://cerebrum-dev.lnkdprod.com/openai/deployments/gpt-4.1/chat/completions?api-version=2024-12-01-preview' \
  --header 'api-key: API_KEY_HERE' \
  --header 'Accept: application/json' \
  --header 'Content-Type: application/json' \
  --data '{
    "model": "gpt-4.1",
    "messages": [
      {
        "role": "system",
        "content": "You are a helpful assistant."
      },
      {
        "role": "user",
        "content": "How are you?"
      }
    ]
  }'
```

In [ ]:
# Simple Chat via HTTP Request
import requests
import json

url = 'https://cerebrum-dev.lnkdprod.com/openai/deployments/gpt-4.1/chat/completions?api-version=2024-12-01-preview'
headers = {
    'api-key': 'API_KEY_HERE',  # Replace with your actual API key
    'Accept': 'application/json',
    'Content-Type': 'application/json'
}
payload = {
    'model': 'gpt-4.1',
    'messages': [
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': 'How are you today?'}
    ]
}
response = requests.post(url, headers=headers, json=payload)
response.json()

## 🛠️ Using Langchain
Now let's integrate a simple tool function and use Cerebrum via Langchain.

Let's create a function to get the LLM using Cerebrum's API. Note that we can leverage several different models, such as `gpt-4.1`, `gpt-4.0`, or `gpt-3.5-turbo`. For this example, we'll use `gpt-4.1`.

*An API key is required in your environment variables.*

In [ ]:
import os
from dotenv import load_dotenv
from langchain_core.messages import AIMessage, ToolMessage
from langchain_core.tools import tool
from langchain_openai import AzureChatOpenAI
from enum import Enum
import httpx

load_dotenv()

class LLMModel(Enum):
    GPT4_1106_PREVIEW = "gpt-4-1106-preview"
    GPT35_TURBO = "gpt-35-turbo"
    GPT35_TURBO_16K = "gpt-35-turbo-16k"
    GPT4 = "gpt-4"
    GPT4_32K = "gpt-4-32k"
    GPT4_VISION_PREVIEW = "gpt-4-vision-preview"
    GPT35_TURBO_1106 = "gpt-35-turbo-1106"
    GPT4_0125_PREVIEW = "gpt-4-0125-Pre view"
    GPT35_TURBO_0125 = "gpt-35-turbo-0125"
    GPT4_TURBO = "gpt-4-turbo"
    GPT4O = "gpt-4o"
    GPT4O_MINI = "gpt-4o-mini"
    O1_preview = "o1-preview"
    O1 = "o1"
    O3_MINI = "o3-mini"
    GPT4_1 = "gpt-4.1"
    GPT4_1_MINI = "gpt-4.1-mini"
    GPT4_1_NANO = "gpt-4.1-nano"

def get_llm(model=LLMModel.GPT4_1):
    return AzureChatOpenAI(
        azure_deployment=model.value,
        api_version="2024-12-01-preview",
        api_key=os.getenv("CEREBRUM_KEY"), # <------- API Key
        azure_endpoint="https://cerebrum-dev.lnkdprod.com/",
        model=model.value,
        model_version="2024-12-01-preview",
        http_client=httpx.Client(),
        http_async_client=httpx.AsyncClient()
    )

Langchain allows the use of tools, which are functions that the LLM can call during a conversation. In this example, we'll create a dummy tool that greets a user.

In [ ]:
@tool(parse_docstring=True)
def dummy_tool(name: str) -> dict:
    """
    A mock tool that returns a greeting.

    Args:
        name (str): The name of the person to greet.
    Returns:
        dict: A greeting message.
    """
    return {"message": f"Hello, {name}!"}

Now let's put it all together. We can bind the tool to the LLM and invoke it in a conversation. Notice that we tell the LLM to use the tool in the user message in plain language, and the LLM will call the tool when appropriate.

We can then see the result of the tool invocation in the response.

In [ ]:
llm = get_llm()
llm_with_tools = llm.bind_tools([dummy_tool])

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Greet Adrian using the dummy tool."}
]

response = llm_with_tools.invoke(messages)
tool_call = response.tool_calls[0]
tool_args = tool_call['args']
tool_call_id = tool_call['id']

tool_output = dummy_tool.invoke(tool_args)

followup_messages = messages + [
    response,
    ToolMessage(tool_call_id=tool_call_id, content=str(tool_output))
]

final_response = llm_with_tools.invoke(followup_messages)
print("Final response:", final_response.content)